In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip instalfrom torch.utils.data import Datasetl torchaudio
    !pip install wandb --upgrade
    # !wandb login
    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews
    !pip install -U ipykernel
    !pip install joypy
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Results')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Figures')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Result')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Jaguas"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import numpy as np
import torchaudio
import torch
import os, sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

!pip install torchvggish==0.1
from torchvggish import vggish, vggish_input

%autoreload 1
# % cd f"{root}/Jaguas/vggish"
# folders = os.listdir(root_path)
# files=[]
# for i in range(len(folders)):
#     path_aux = "{}/{}".format(root_path, folders[i])
#     files += list(Path(path_aux).rglob("*.{}".format("wav")))
# file = str(files[16])

UsageError: Line magic function `%autoreload` not found.


In [3]:
def find_pickle_filenames( path_to_dir, suffix=".pickle" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

folders = os.listdir(root_path)

In [7]:
root_audios = f"{root}/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
df_audios = pd.read_csv(root_audios)
df_audios.drop(columns = ["Unnamed: 0"], inplace=True)

In [6]:
X = []
y = []
for i in range(len(df_audios)):
    embedding_model = vggish()
#     embedding_model = embedding_model.to("cuda")
    embedding_model.eval()
    if (i%100) == 0:
        print(i)
    else:
        pass
    try:
        file = root_path+"/"+df_audios.iloc[i].loc["Recorder"]+"_m/"+df_audios.loc[i, "Filename"]
        audio_sample = vggish_input.wavfile_to_examples(file)
        embeddings = embedding_model.forward(audio_sample)
    except:
        print(f"Error in {file}")
        continue
    X.append(embeddings.detach())
    y.append(file)
torch.save(X, f"Features_vggish/vggish_features.pth")
torch.save(y, f"Features_vggish/vggish_filenames.pth")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [29]:
X=torch.load(f"Features_vggish/vggish_features.pth", map_location="cpu")

In [30]:
for i in range(len(X)):
    X[i] = X[i].reshape(X[i].shape[0]*X[i].shape[1])

In [ ]:
# Initialise model and download weights
embedding_model = vggish()
embedding_model.eval()

example = vggish_input.wavfile_to_examples(root_path+"/"+folders[0]+"/"+df_audios.loc[0, "Filename"])
embeddings = embedding_model.forward(example)
embeddings.detach().numpy().shape
plt.imshow(embeddings.detach().numpy())

In [ ]:
from torch.utils.data import DataLoader
root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
dataset_test = torch.load(f'{root}/temporal/dataset_test_ae_jaguas_new.pth')
dataset_train = torch.load(f'{root}/temporal/dataset_train_ae_jaguas_new.pth')
training_loader = DataLoader(dataset_train, batch_size=1)
test_loader = DataLoader(dataset_test, batch_size=1)
iterator = iter(training_loader)

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []

In [ ]:
a = np.ones([1,20])
b = np.zeros([1,4])
c =np.append(a,b, axis=1)
c

In [ ]:
def fileToRecord(file, audio_lenght=12, resampling=22050, original_length=60):
    record, sr = torchaudio.load(file)
    audio_len = audio_lenght*resampling
    record = torch.mean(record, dim=0, keepdim=True)
    record = torchaudio.transforms.Resample(sr, resampling)(record)
    missing_padding = resampling * original_length - record.shape[1]
    padding = torch.zeros([1, missing_padding])
    record = torch.cat((record, padding), axis=1)
    print(record.shape[1]/resampling, missing_padding)
    record = record[:, :audio_len*(record.shape[1] // audio_len)]
    record = torch.reshape(record, (record.shape[1] // audio_len, audio_len))
    print(record.shape)
    record = record[0]
    return record
    

In [ ]:
record = fileToRecord(file)

In [ ]:
(b==record).all()

In [ ]:
for id, item in enumerate(dataset_train):
    index = training_loader.dataset.indices[id]
    file = str(files[index])
    record = fileToRecord(file)


    print(item.index)

In [ ]:
training_loader.dataset.indices

In [ ]:
for id, item in enumerate(dataset_train.dataset.):
    if (id+1)% 5 == 0:
        print("entro")
        break
    if id% 500 == 0:
        print(f"id: {id + 1} of {len(dataset_train)}")
    try:
        spec, wav, label = next(iterator)
    except:
        print(f"iterator broken in index {id}")
        continue
    wav = wav.unsqueeze(0)
    with torch.no_grad():
        codes = model.encoder(wav)
    print(codes.shape)
    encodings = torch.reshape(codes,(codes.shape[0], codes.shape[1]*codes.shape[2]))
    print(encodings.shape)
    training_samples_list_torch[id] = encodings[0]
    training_recorder_list.append(label["recorder"])
    training_hour_list.append(label["hour"])
    training_minute_list.append(label["minute"])

print(f"------------------------------------\n{id}\nProcess finished\nSaving data\n------------------------------------")
torch.save(training_samples_list_torch, "training_samples_list_torch_vggish_feats.pth")
torch.save(training_recorder_list, "training_recorder_list_vggish_feats.pth")
torch.save(training_hour_list, "training_hour_list_vggish_feats.pth")
torch.save(training_minute_list, "training_minute_list_vggish_feats.pth")
training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
torch.save(training_labels_list, "training_labels_list_vggish_feats.pth")